In [15]:
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [5]:
# rank,name,rating,url,location_coordinate,location_country,location_region,elevation_top_m,elevation_difference_m,total_slope_length_km,number_of_lifts,number_of_slopes,annual_snowfall_cm,number_of_matches

ski_dataset = MyPyTable().load_from_file("./input_data/ski-resorts.csv")

ski_X = ski_dataset.get_columns(["elevation_top_m", "number_of_slopes", "annual_snowfall_cm"])
ski_y = ski_dataset.get_column("rating")

# TODO: make sure this is the right order
matrix_headers = ["rating", "very high", "high", "above average", "average", "below average", "low", "very low", "Total", "Recognition (%)"]

discretizer = myutils.discretize_ranking

In [53]:
# TODO: modify categorical to be a list given
knn_clf = MyKNeighborsClassifier()

knn_actual, knn_pred, knn_accuracy, knn_error = myevaluation.cross_val_predict\
    (ski_X, ski_y, knn_clf, discretizer=discretizer, n_splits=10, shuffle=True, stratify=False)


print(knn_actual)
print(knn_pred)

print(f"Accuracy: {knn_accuracy}, Error Rate: {knn_error}")
print()

matrix = myevaluation.confusion_matrix(knn_actual, knn_pred, matrix_headers[1:8])
print(myevaluation.tabulate_confusion_matrix(matrix, headers=matrix_headers))



['below average', 'low', 'average', 'low', 'average', 'low', 'below average', 'below average', 'low', 'low', 'low', 'low', 'below average', 'low', 'low', 'below average', 'average', 'average', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'below average', 'below average', 'average', 'low', 'below average', 'average', 'low', 'very low', 'average', 'low', 'low', 'low', 'low', 'average', 'below average', 'very low', 'average', 'below average', 'below average', 'very low', 'low', 'below average', 'average', 'below average', 'average', 'low', 'low', 'below average', 'below average', 'below average', 'average', 'low', 'below average', 'low', 'low', 'below average', 'low', 'below average', 'low', 'below average', 'below average', 'below average', 'very low', 'below average', 'below average', 'below average', 'low', 'below average', 'below average', 'low', 'low', 'low', 'below average', 'below average', 'below average', 'below average', 'very low', 'below average', 'low', 'low', 'low', 'bel

In [ ]:
# TODO: ask sprint where I should implement part 4 of random forest
forest_clf = MyRandomForestClassifier()


"""
 figuring out why sometimes nothing is predicted
"""

y_disc = [discretizer(y) for y in ski_y]

X_test, y_test = forest_clf.fit(ski_X, y_disc, 5, 3, 2)
forest_pred = forest_clf.predict(X_test)
# forest_actual, forest_pred, forest_accuracy, forest_error = myevaluation.cross_val_predict\
#     (ski_X, ski_y, forest_clf, discretizer=discretizer, n_splits=10, shuffle=True, stratify=False)
print(y_test[:10])
print(forest_pred[:10])

forest_accuracy = myevaluation.accuracy_score(y_test, forest_pred)
forest_error = 1 - forest_accuracy

forest_actual = y_test
""""""



print(f"Accuracy: {forest_accuracy}, Error Rate: {forest_error}")
print()

matrix = myevaluation.confusion_matrix(forest_actual, forest_pred, matrix_headers[1:8])
print(myevaluation.tabulate_confusion_matrix(matrix, headers=matrix_headers))

['', 'above average', '', 'below average', 'average', '', 'average', '', '', 'average']
['', 'average', '', 'below average', 'average', '', 'average', '', '', 'average']
['', 'average', '', 'below average', 'average', '', 'average', '', '', 'average']
['very high', 'high', 'above average', 'above average', 'above average', 'above average', 'above average', 'above average', 'above average', 'average']
['', 'average', '', 'below average', 'average', '', 'average', '', '', 'average']
Accuracy: 0.6011029411764706, Error Rate: 0.39889705882352944



KeyError: ''